In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

In [3]:

sub_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/sample_submission.csv')
train_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/train.csv')
test_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/test.csv')

In [4]:
train_df.head()

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9
1,1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8
2,2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9
3,3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11
4,4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8


In [5]:
train_df.isnull().any()

id                False
Sex               False
Length            False
Diameter          False
Height            False
Weight            False
Shucked Weight    False
Viscera Weight    False
Shell Weight      False
Age               False
dtype: bool

## handling the sex column

In [6]:
train_sex_col = pd.get_dummies(train_df['Sex'], prefix='Sex', prefix_sep='_', drop_first=True)
test_sex_col = pd.get_dummies(test_df['Sex'], prefix='Sex', prefix_sep='_', drop_first=True)

In [7]:
train_sex_col.head()

,Sex_I,Sex_M
0,1,0
1,1,0
2,0,1
3,0,0
4,1,0


In [8]:
train_df=pd.concat([train_df,train_sex_col],axis=1)
test_df=pd.concat([test_df,test_sex_col],axis=1)

    

In [9]:
train_df.drop(['id','Sex'],axis=1, inplace=True)
test_df.drop(['id','Sex'],axis=1, inplace=True)


In [10]:
train_df.head()

,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age,Sex_I,Sex_M
0,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9,1,0
1,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8,1,0
2,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9,0,1
3,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11,0,0
4,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8,1,0


In [11]:
train_df.columns

Index(['Length', 'Diameter', 'Height', 'Weight', 'Shucked Weight',
       'Viscera Weight', 'Shell Weight', 'Age', 'Sex_I', 'Sex_M'],
      dtype='object')

# Feature Enginnering

In [12]:
def generate_features(data):
  df_ = data.copy()
  df_['Shell Weight'] = np.where(df_['Shell Weight'] > df_['Weight'],df_['Weight'],df_['Shell Weight'])
  df_['Viscera Weight'] = np.where(df_['Viscera Weight'] > df_['Weight'],df_['Weight'],df_['Viscera Weight'])
  df_['Shucked Weight'] = np.where(df_['Shucked Weight'] > df_['Weight'],df_['Weight'],df_['Shucked Weight'])

  return df_
new_train = generate_features(train_df)
new_test = generate_features(test_df)

# Modeling

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
from sklearn.svm import SVR

In [14]:
X=new_train.drop('Age', axis=1)
y=new_train['Age']

In [15]:
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.8,random_state=42)

In [16]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(59240, 9)
(59240,)
(14811, 9)
(14811,)


In [24]:
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout



# Define the deep learning model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))

# Compile the model
model.compile(optimizer=Adam(), loss='mean_absolute_error')

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)

# Train the model with early stopping
model.fit(X_train, y_train, batch_size=32, epochs=50, verbose=1, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
print("Mean Squared Error:", mae)

Epoch 1/50
1852/1852 [==============================] - 5s 2ms/step - loss: 1.8176 - val_loss: 1.4225
Epoch 2/50
1852/1852 [==============================] - 6s 3ms/step - loss: 1.6044 - val_loss: 1.4509
Epoch 3/50
1852/1852 [==============================] - 4s 2ms/step - loss: 1.5674 - val_loss: 1.4784
Epoch 4/50
1852/1852 [==============================] - 4s 2ms/step - loss: 1.5482 - val_loss: 1.4098
Epoch 5/50
1852/1852 [==============================] - 6s 3ms/step - loss: 1.5241 - val_loss: 1.5277
Epoch 6/50
1852/1852 [==============================] - 5s 3ms/step - loss: 1.5132 - val_loss: 1.4422
Epoch 7/50
1852/1852 [==============================] - 4s 2ms/step - loss: 1.5069 - val_loss: 1.4454
Epoch 8/50
1852/1852 [==============================] - 6s 3ms/step - loss: 1.5033 - val_loss: 1.4038
Epoch 9/50
1852/1852 [==============================] - 4s 2ms/step - loss: 1.4946 - val_loss: 1.4091
Epoch 10/50
1852/1852 [==============================] - 4s 2ms/step - loss: 1.493

# Prediction

**Based on the above result svr yeilds the better results**

In [25]:

# Make predictions on the scaled test data using the best model
y_test_pred = model.predict(test_df)

# Print the predicted values
print("Predicted Age for Test Data:")
print(y_test_pred)

1543/1543 [==============================] - 2s 1ms/step
Predicted Age for Test Data:
[[ 6.9867496]
 [ 7.999961 ]
 [10.087788 ]
 ...
 [12.37459  ]
 [10.056418 ]
 [11.496449 ]]


In [26]:
test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/test.csv')

In [27]:
y_test_pred = np.squeeze(y_test_pred)

submission_df = pd.DataFrame({'id': test['id'], 'Age': y_test_pred})
submission_df.to_csv('submission.csv', index=False)

In [28]:
submission_df

,id,Age
0,74051,6.986750
1,74052,7.999961
2,74053,10.087788
3,74054,9.565274
4,74055,7.479093
...,...,...
49363,123414,8.783352
49364,123415,7.529817
49365,123416,12.374590
49366,123417,10.056418
